In [28]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from scipy.io import loadmat

## Loading Data

In [29]:
data = loadmat('data/ex4data1.mat')

y = data['y']
print('y.shape = ',y.shape)

# Add intercept
X = data['X']
print('X.shape : ', X.shape)
X = np.c_[np.ones((data['X'].shape[0],1)), data['X']]
print('X.shape : ', X.shape, ' intercept added.')

y.shape =  (5000, 1)
X.shape :  (5000, 400)
X.shape :  (5000, 401)  intercept added.


In [30]:
weights = loadmat('data/ex4weights.mat')
theta1, theta2 = weights['Theta1'], weights['Theta2']
print('theta1 :', theta1.shape)                             # Input size : 401 including bias
                                                            # Num of hidden units : 10
print('theta2 :', theta2.shape)                             # Num of lables : 10
params = np.r_[theta1.ravel(), theta2.ravel()]
print('params :', params.shape)

theta1 : (25, 401)
theta2 : (10, 26)
params : (10285,)


## Activation Function

In [31]:
def sigmoid(z):
    return(1 / (1 + np.exp(-z)))

def sigmoidGradient(z):
    return(sigmoid(z)*(1-sigmoid(z)))

## Cost Function & Gradient

In [32]:
def nnCostFunction(nn_params, input_layer_size, hidden_layer_size, num_labels, features, classes, reg):
    
    theta1 = nn_params[0:(hidden_layer_size*(input_layer_size+1))].reshape(hidden_layer_size,(input_layer_size+1))
    # 25 x 401
    theta2 = nn_params[(hidden_layer_size*(input_layer_size+1)):].reshape(num_labels,(hidden_layer_size+1))
    # 10 x 26 
    
    m = features.shape[0]
    y_matrix = pd.get_dummies(classes.ravel()).as_matrix() 
    
    
    ## Cost
    a1 = features                                                 # 5000 x 401
        
    z2 = theta1.dot(a1.T)                                         # 25x401 * 401x5000 = 25 x 5000 
    a2 = np.c_[np.ones((features.shape[0],1)),sigmoid(z2.T)]      # 5000 x 26 
    
    z3 = theta2.dot(a2.T)                                         # 10x26 * 26x5000 = 10 x 5000 
    a3 = sigmoid(z3)                                              # 10 x 5000
    print(a3.T[1000,:])
    
    J = - 1 * (1/m) * np.sum( ( np.log(a3.T)*(y_matrix) + np.log(1-a3).T*(1-y_matrix) ) ) + \
        (reg/(2*m)) * ( np.sum(np.square(theta1[:,1:])) + np.sum(np.square(theta2[:,1:])) )

    
    ## Gradients
    d3 = a3.T - y_matrix                                         # 5000 x 10
    d2 = theta2[:,1:].T.dot(d3.T)*sigmoidGradient(z2)            # 25x10 *10x5000 * 25x5000 = 25 x 5000
    
    delta1 = d2.dot(a1)                                          # 25x5000 * 5000x401 = 25 x 401
    delta2 = d3.T.dot(a2)                                        # 10x5000 *5000x26 = 10 x 26
    
    theta1_ = np.c_[np.ones((theta1.shape[0],1)),theta1[:,1:]]
    theta2_ = np.c_[np.ones((theta2.shape[0],1)),theta2[:,1:]]
    
    theta1_grad = delta1/m + (theta1_*reg)/m
    theta2_grad = delta2/m + (theta2_*reg)/m
    
    return(J, theta1_grad, theta2_grad)


In [33]:
input_layer_size = 400
hidden_layer_size = 25
num_labels = 10
features = X
classes = y
reg = 0
nnCostFunction(params, input_layer_size, hidden_layer_size, num_labels, X, y, reg)[0]

[3.85613838e-04 9.68544083e-01 1.92134752e-03 1.38526834e-04
 3.20810992e-03 7.01713717e-04 6.45235792e-04 1.66993752e-02
 1.00700486e-01 3.25396474e-03]


C:\Users\USMEJOS\Downloads\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if __name__ == '__main__':


0.2876291651613189

In [34]:
params.shape

(10285,)